In [103]:
# import necessary libaries and data files
import numpy as np
from data import datas
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [104]:
#load data and manupulate datas if needed
bus_data, line_data, load_data, gen_data, wind_data = datas() #data loading

line_data = pd.DataFrame(line_data).sort_values(by=[0,1]).values #line data sorting and data type change

bus_data = pd.DataFrame(bus_data).sort_values(by = [0]).values #Bus data sorting and data type change

load_data = pd.DataFrame(load_data).sort_values(by = [0]).astype({0:'int'}).set_index(0) #Load data sorting and data type change

gen_data = pd.DataFrame(gen_data).sort_values(by = [0]).astype({0:'int'}).set_index(0) #Generation data sorting and data type change

wind_data = pd.DataFrame(wind_data).sort_values(by = [0]).astype({0:'int'}).set_index(0) #Wind Generation data sorting and data type change

In [105]:
base_mva = 100
R = line_data[:, 2]
X = line_data[:, 3]
B = 0.5j * line_data[:, 4]
taps = line_data[:,5]
Bsh = 1j* bus_data[:, 3]
Z = R + 1j * X
Y = 1 / Z
nline = len(line_data[:, 0])
nbus = int(np.amax(line_data[:, :2]))  # Total Number of buses

In [106]:
def tap_handle(x):
    if np.real(x) == 0:
        return 1
    else:
        return complex(x)
    
# Y Bus Formation
def Y_bus():
    Ybus = np.zeros((nbus, nbus), dtype=complex)
    Y1 = np.zeros((73, 73), dtype=complex)
    np.fill_diagonal(Y1,Bsh)
    
    for k in range(nline):
        # Off Diagonal Elements
        Ybus[int(line_data[k, 0]) - 1, int(line_data[k, 1]) - 1] -= (Y[k])/np.conj(tap_handle(taps[k]))
        Ybus[int(line_data[k, 1]) - 1, int(line_data[k, 0]) - 1] -= (Y[k])/tap_handle(taps[k])

        # Diagonal Elements
        Ybus[int(line_data[k, 0]) - 1, int(line_data[k, 0]) - 1] += (Y[k] + B[k])/pow(abs(tap_handle(taps[k])),2)
        Ybus[int(line_data[k, 1]) - 1, int(line_data[k, 1]) - 1] += Y[k] + B[k]
    
    Y_bus_ = Ybus + Y1
    return Y_bus_

In [107]:
#From Bus Data
bus_no = bus_data[:, 0].astype(int)
bus_type = bus_data[:,1].astype(int)

#From Load Data
pl = np.zeros(nbus)
pf = np.zeros(nbus)
delta = np.zeros(nbus)
for k in load_data.index:
    pl[k-1] = load_data.loc[k, 2]/base_mva #base power
    pf[k-1] = load_data.loc[k, 1]
ql = pl * np.tan(np.arccos(pf))

#From Generation Data
pg_ = np.zeros(nbus)
qg = np.zeros(nbus)
vmag = np.ones(nbus)
qmin = np.zeros(nbus)
qmax = np.zeros(nbus)
for k in gen_data.index:
    pg_[k-1] = (gen_data.loc[k, 4]).sum()/base_mva
    vmag[k-1] = np.mean(gen_data.loc[k, 3])
    qmin[k-1] = gen_data.loc[k, 2].sum()
    qmax[k-1] = gen_data.loc[k, 1].sum()

# From Wind Data
pwg = np.zeros(nbus)
for k in wind_data.index:
    pwg[k-1] = (wind_data.loc[k, 4])/base_mva

pg = pg_ + pwg
b_data = pd.DataFrame()
b_data['Bus_No'] = bus_no
b_data['Bus_type'] = bus_type
b_data['Pg'] = pg
b_data['Qg'] = qg
b_data['Pl'] = pl
b_data['Ql'] = ql
b_data['Vmag'] = vmag
b_data['Delta'] = delta
b_data['Qmin'] = qmin/base_mva
b_data['Qmax'] = qmax/base_mva
b_data['Vr'] = vmag * np.cos(delta)
b_data['Vm'] = vmag * np.sin(delta)
b_data = b_data.set_index('Bus_No')
b_data.head()

,Bus_type,Pg,Qg,Pl,Ql,Vmag,Delta,Qmin,Qmax,Vr,Vm
Bus_No,,,,,,,,,,,
1,2,0.5282,0.0,0.5635,0.114718,1.035,0.0,-0.5,0.8,1.035,0.0
2,1,0.0000,0.0,0.5041,0.103934,1.035,0.0,-0.5,0.8,1.035,0.0
3,1,0.0000,0.0,0.9342,0.192128,1.000,0.0,0.0,0.0,1.000,0.0
4,1,0.0000,0.0,0.3855,0.078077,1.000,0.0,0.0,0.0,1.000,0.0
5,1,0.0000,0.0,0.3707,0.073113,1.000,0.0,0.0,0.0,1.000,0.0


In [108]:
def d_Ir_Vr(Vr, Vm, P, Q):
    V = pow(Vr,2) + pow(Vm,2)
    return (P* (pow(Vm, 2) - pow(Vr, 2)) - 2*Q*Vr*Vm)/pow(V,2)

def d_Ir_Vm(Vr, Vm, P, Q):
    V = pow(Vr,2) + pow(Vm,2)
    return (Q* (pow(Vr, 2) - pow(Vm, 2)) - 2*P*Vr*Vm)/pow(V,2)

def d_Im_Vr(Vr, Vm, P, Q):
    V = pow(Vr,2) + pow(Vm,2)
    return (Q* (pow(Vr, 2) - pow(Vm, 2)) - 2*P*Vr*Vm)/pow(V,2)

def d_Im_Vm(Vr, Vm, P, Q):
    V = pow(Vr,2) + pow(Vm,2)
    return (P* (pow(Vr, 2) - pow(Vm, 2)) - 2*Q*Vr*Vm)/pow(V,2)

def d_Ir_Q(Vr, Vm):
    V = pow(Vr,2) + pow(Vm,2)
    return Vm/pow(V,2)

def d_Im_Q(Vr, Vm):
    V = pow(Vr,2) + pow(Vm,2)
    return -Vr/pow(V,2)



In [109]:
def data_df(b_data_):
    b_df = b_data_
    b_df['Psh'] = b_data['Pg'] - b_data['Pl']
    b_df['Qsh'] = b_data['Qg'] - b_data['Ql']
    b_df['Ir'] = (b_df['Psh']*b_df['Vr'] + b_df['Qsh']*b_df['Vm'])/(pow(b_df['Vr'], 2) + pow(b_df['Vm'], 2))
    b_df['Im'] = (b_df['Psh']*b_df['Vm'] - b_df['Qsh']*b_df['Vr'])/(pow(b_df['Vr'], 2) + pow(b_df['Vm'], 2))
    b_df['d_Ir_Vr'] = d_Ir_Vr(b_df['Vr'],b_df['Vm'], b_df['Psh'],b_df['Qsh'])
    b_df['d_Ir_Vm'] = d_Ir_Vm(b_df['Vr'],b_df['Vm'], b_df['Psh'],b_df['Qsh'])
    b_df['d_Im_Vr'] = d_Im_Vr(b_df['Vr'],b_df['Vm'], b_df['Psh'],b_df['Qsh'])
    b_df['d_Im_Vm'] = d_Ir_Vm(b_df['Vr'],b_df['Vm'], b_df['Psh'],b_df['Qsh'])
    b_df['d_Ir_Q']  = d_Ir_Q(b_df['Vr'],b_df['Vm'])
    b_df['d_Im_Q']  = d_Im_Q(b_df['Vr'],b_df['Vm'])


    con_pv = (b_df['Bus_type'] == 2) # Condition for PV Bus
    con_pq = (b_df['Bus_type'] == 1)
    b_df.loc[con_pq,'delta_IR'] = (- b_df.loc[con_pq,'d_Ir_Vr']*b_df.loc[con_pq,'Vr'] - 
                                   b_df.loc[con_pq,'d_Ir_Vm']*b_df.loc[con_pq,'Vm'] + b_df.loc[con_pq,'Ir'])
    b_df.loc[con_pq,'delta_IM'] = (- b_df.loc[con_pq,'d_Im_Vr']*b_df.loc[con_pq,'Vr'] - 
                                   b_df.loc[con_pq,'d_Im_Vm']*b_df.loc[con_pq,'Vm'] + b_df.loc[con_pq,'Im'])

    b_df.loc[con_pv,'delta_IR'] = (- b_df.loc[con_pv,'d_Ir_Vr']*b_df.loc[con_pv,'Vr'] - 
                                   b_df.loc[con_pv,'d_Ir_Vm']*b_df.loc[con_pv,'Vm'] - 
                                   b_df.loc[con_pv,'d_Ir_Q']*b_df.loc[con_pv,'Qsh'] + b_df.loc[con_pv,'Ir'])
    b_df.loc[con_pv,'delta_IM'] = (- b_df.loc[con_pv,'d_Im_Vr']*b_df.loc[con_pv,'Vr'] - 
                                   b_df.loc[con_pv,'d_Im_Vm']*b_df.loc[con_pv,'Vm'] - 
                                   b_df.loc[con_pv,'d_Im_Q']*b_df.loc[con_pv,'Qsh'] + b_df.loc[con_pv,'Im'])

    return b_df

In [110]:
data_df(b_data)

,Bus_type,Pg,Qg,Pl,Ql,Vmag,Delta,Qmin,Qmax,Vr,...,Ir,Im,d_Ir_Vr,d_Ir_Vm,d_Im_Vr,d_Im_Vm,d_Ir_Q,d_Im_Q,delta_IR,delta_IM
Bus_No,,,,,,,,,,,,,,,,,,,,,
1,2,0.5282,0.0,0.5635,0.114718,1.035,0.0,-0.50,0.80,1.035,...,-0.034106,0.110839,0.032953,-0.107091,-0.107091,-0.107091,0.0,-0.901943,-0.068213,0.118208
2,1,0.0000,0.0,0.5041,0.103934,1.035,0.0,-0.50,0.80,1.035,...,-0.487053,0.100419,0.470583,-0.097023,-0.097023,-0.097023,0.0,-0.901943,-0.974106,0.200838
3,1,0.0000,0.0,0.9342,0.192128,1.000,0.0,0.00,0.00,1.000,...,-0.934200,0.192128,0.934200,-0.192128,-0.192128,-0.192128,0.0,-1.000000,-1.868400,0.384256
4,1,0.0000,0.0,0.3855,0.078077,1.000,0.0,0.00,0.00,1.000,...,-0.385500,0.078077,0.385500,-0.078077,-0.078077,-0.078077,0.0,-1.000000,-0.771000,0.156154
5,1,0.0000,0.0,0.3707,0.073113,1.000,0.0,0.00,0.00,1.000,...,-0.370700,0.073113,0.370700,-0.073113,-0.073113,-0.073113,0.0,-1.000000,-0.741400,0.146226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,2,4.0000,0.0,0.0000,0.000000,1.050,0.0,-0.50,2.00,1.050,...,3.809524,0.000000,-3.628118,0.000000,0.000000,0.000000,0.0,-0.863838,7.619048,0.000000
70,1,0.0000,0.0,0.0000,0.000000,1.050,0.0,-0.60,0.96,1.050,...,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.0,-0.863838,0.000000,0.000000
71,2,0.5424,0.0,0.0000,0.000000,1.050,0.0,-1.25,3.10,1.050,...,0.516571,0.000000,-0.491973,0.000000,0.000000,0.000000,0.0,-0.863838,1.033143,0.000000


In [111]:
def Jacobian(df, y_bus):
    b_df = data_df(df)
    G = np.real(y_bus)
    B = np.imag(y_bus)
    delta_I = np.zeros((nbus, 1, 3, 1))
    J = np.zeros((nbus, nbus, 3, 3))
    for i in b_df.index:
        for j in b_df.index:
            if b_df.loc[i,"Bus_type"] != 3:
                if b_df.loc[i,"Bus_type"] == 1: # For PQ Bus
                    if i == j:
                        J[i-1, j-1][0, 0] = -b_df.loc[i, 'd_Im_Vr'] + B[i-1, i-1]
                        J[i-1, j-1][0, 1] = -b_df.loc[i, 'd_Im_Vm'] + G[i-1, i-1]
                        J[i-1, j-1][1, 0] = -b_df.loc[i, 'd_Ir_Vr'] + G[i-1, i-1]
                        J[i-1, j-1][1, 1] = -b_df.loc[i, 'd_Ir_Vm'] - B[i-1, i-1]

                    else:
                        J[i-1, j-1][0, 0] =  B[i-1, j-1]
                        J[i-1, j-1][0, 1] =  G[i-1, j-1]
                        J[i-1, j-1][1, 0] =  G[i-1, j-1]
                        J[i-1, j-1][1, 1] =  -B[i-1, j-1]
                    
                    delta_I[i-1,0][0,0] = b_df.loc[i,'delta_IM']
                    delta_I[i-1,0][1,0] = b_df.loc[i,'delta_IR']
                   
                        
                else: #For PV Bus
                    if i == j:
                        J[i-1, j-1][0, 0] = -b_df.loc[i, 'd_Im_Vr'] + B[i-1, i-1]
                        J[i-1, j-1][0, 1] = -b_df.loc[i, 'd_Im_Vm'] + G[i-1, i-1]
                        J[i-1, j-1][1, 0] = -b_df.loc[i, 'd_Ir_Vr'] + G[i-1, i-1]
                        J[i-1, j-1][1, 1] = -b_df.loc[i, 'd_Ir_Vm'] - B[i-1, i-1]
                        J[i-1, j-1][0, 2] = -b_df.loc[i, 'd_Im_Q' ]
                        J[i-1, j-1][1, 2] = -b_df.loc[i, 'd_Ir_Q' ]
                        J[i-1, j-1][2, 0] = b_df.loc[i, 'Vr']
                        J[i-1, j-1][2, 1] = b_df.loc[i, 'Vm']
                    
                    else:
                        J[i-1, j-1][0, 0] =  B[i-1, j-1]
                        J[i-1, j-1][0, 1] =  G[i-1, j-1]
                        J[i-1, j-1][1, 0] =  G[i-1, j-1]
                        J[i-1, j-1][1, 1] =  -B[i-1, j-1]

                    delta_I[i-1,0][0,0] = b_df.loc[i,'delta_IM']
                    delta_I[i-1,0][1,0] = b_df.loc[i,'delta_IR']
                    delta_I[i-1,0][2,0] = pow(b_df.loc[i, 'Vmag'],2)
        
    return delta_I, J


In [112]:
I, J = Jacobian(b_data, Y_bus())

In [113]:
J[12,12]

array([[-64.89963453,   8.57519635,   0.        ],
       [  6.9799074 ,  65.43930892,   0.        ],
       [  0.        ,   0.        ,   0.        ]])

In [114]:
I[0,0]

array([[ 0.1182083 ],
       [-0.06821256],
       [ 1.071225  ]])

In [115]:
for k in b_data.index:
    print(k)
    np.linalg.inv(J[k-1,k-1])

1
2


LinAlgError: Singular matrix